In [4]:
# library
import re

import jaconv
import pykakasi
from kanjize import kanji2number


class KanjiTransfer:
    """漢字からの各種変換を担当します

    以下プロセス行い漢字を半角カナに変換する
        1.漢数字をアラビア数字に変換(ただし漢数字を含む固有名詞は対象外)
        2.㈱たちを取り除く
        3.全角[]に囲まれた範囲をカナ変換対象外とする
        4.カナ変換
        5.拗音を通常カナに変換する
        6.カナ -> 半角カナ変換

    Examples:
        >>> kanji = KanjiTransfer()
        >>> kanji.convert_kanji_to_hankaku_kana(target_string_kanji)

    Notes:
        漢数字を含む固有名詞は _proer_nouns()関数内にlistとして定義してください

    """

    def __init__(self):
        self._proper_nouns = self._proper_nouns()

    def _proper_nouns(self) -> list:
        """漢数字を含む固有名詞定義,アラビア数字変換しない"""
        return [
            '第一',
            '第二',
            ]

    def _convert_fullwidth_to_halfwidth_digits(self, string:str) -> str:
        """全角漢数字を半角数字に変換、ただし固有名詞は除く"""
        # 固有名詞/その他分割するためのパターン文字生成
        pattern = '|'.join(self._proper_nouns)

        # 固有名詞とその他部分に分割しリスト格納
        matches = list(re.finditer(pattern, string))
        converted_parts = []
        prev_end = 0
        for match in matches:
            start, end = match.span()
            converted_parts.append(string[prev_end:start])
            converted_parts.append(match.group())
            prev_end = end
        converted_parts.append(string[prev_end:])

        # 固有名詞部分以外は全角漢数字を半角数字に変換する
        converted_parts = [re.sub(r'([一二三四五六七八九十]+)', self._kanji_to_arabic, part) if part not in self._proper_nouns else part for part in converted_parts]

        # 変換後部品群を結合して返す
        return ''.join(converted_parts)

    def _kanji_to_arabic(self, matchobj) -> str:
        """漢数字を半角数字に変換する"""
        # 漢数字をアラビア数字に変換する
        kanji_num_str = matchobj.group(1)
        arabic_num_str = str(kanji2number(kanji_num_str))

        # 全角数字を半角に変換
        return jaconv.z2h(arabic_num_str, digit=True)

    def _remove_corporation_symbol(self, string: str) -> str:
        """㈱を取り除く"""
        # TODO(me) ㈱パターンもいくつかある可能性、株式会社や株というケース、現物見て考察する
        _string = string
        _string = re.sub(r'株式会社','', _string)
        _string =  re.sub(r'㈱','', _string)
        _string =  re.sub(r'（株）','', _string)  # noqa: RUF001  編集対象文字であり許容
        return _string                            # noqa: RET504  指摘対応すると保守時のメンテナンス性が下落するため許容

    def _remove_braketed_prefix(self, string: str) -> str:
        """全角[]に囲まれた範囲を削る,カナ変換結果に還元しない"""
        return re.sub(r'^\［.*\］','', string)    # noqa: RUF001  編集対象文字であり許容

    def _convert_to_kana(self, string: str) -> str:
        """カナ変換"""
        kakasi = pykakasi.kakasi()
        return ''.join([item['kana'] for item in kakasi.convert(string)])

    def _convert_yoon_to_regular_kana(self, string: str) -> str:
        """拗音を通常カナに変換する"""
        yoon_mapping = {
            'キャ': 'キヤ',
            'キュ': 'キユ',
            'キョ': 'キヨ',
            'シャ': 'シヤ',
            'シュ': 'シユ',
            'ショ': 'シヨ',
            'ティ': 'テイ',
            'チャ': 'チヤ',
            'チュ': 'チユ',
            'チョ': 'チヨ',
            'ニャ': 'ニヤ',
            'ニュ': 'ニユ',
            'ニョ': 'ニヨ',
            'ヒャ': 'ヒヤ',
            'ヒュ': 'ヒユ',
            'ヒョ': 'ヒヨ',
            'ミャ': 'ミヤ',
            'ミュ': 'ミユ',
            'ミョ': 'ミヨ',
            'リャ': 'リヤ',
            'リュ': 'リユ',
            'リョ': 'リヨ',
            'ギャ': 'ギヤ',
            'ギュ': 'ギユ',
            'ギョ': 'ギヨ',
            'ジャ': 'ジヤ',
            'ジュ': 'ジユ',
            'ジョ': 'ジヨ',
            'ビャ': 'ビヤ',
            'ビュ': 'ビユ',
            'ビョ': 'ビヨ',
            'ピャ': 'ピヤ',
            'ピュ': 'ピユ',
            'ピョ': 'ピヨ',
        }
        # マッピングを使用して拗音を通常カナに変換
        for yoon, regular in yoon_mapping.items():
            string = string.replace(yoon, regular)
        return string


    def _convert_kana_to_hankaku_kana(self, string: str) -> str:
        """カナ->半角カナ変換"""
        return jaconv.z2h(string, ascii=True, digit=True, kana=True)


    def convert_kanji_to_hankaku_kana(self, target_str: str) -> str:
        """漢字を半角カナに変換する"""
        try:
            # 1.漢数字をアラビア数字に変換(ただし漢数字を含む固有名詞は対象外)
            # 2.㈱たちを取り除く
            # 3.全角[]に囲まれた範囲をカナ変換対象外とする
            # 4.カナ変換
            # 5.拗音を通常カナに変換する
            # 6.カナ -> 半角カナ変換
            _str = self._convert_fullwidth_to_halfwidth_digits(target_str)
            _str = self._remove_corporation_symbol(_str)
            _str = self._remove_braketed_prefix(_str)
            _str = self._convert_to_kana(_str)
            _str = self._convert_yoon_to_regular_kana(_str)
            _str = self._convert_kana_to_hankaku_kana(_str)

        except Exception as e:
            print(f'全角漢字→半角カナ変換に失敗しました: {target_str}')
            print(str(e))

        return _str

In [6]:
# 変換検証
target_string_kanji = '［㈱企業第一アンド］㈱第二アンド第三株式会社かな変換はチョー大変'

kanji = KanjiTransfer()
kanji.convert_kanji_to_hankaku_kana(target_string_kanji)

'ﾀﾞｲﾆｱﾝﾄﾞﾀﾞｲ3ｶﾅﾍﾝｶﾝﾊﾁﾖｰﾀｲﾍﾝ'